# **MLOps: Weights and Biases with Keras (CIFAR)**

### Dr. Radhika Chapaneri



What this notebook covers:

Integrate Weights & Biases with Keras code to add experiment tracking to pipeline.

1. Storing hyperparameters and metadata in a `config`.
2. Passing the wandb Keras callbacks to `model.fit`. This will automatically log training metrics, like loss, and system metrics, like GPU and CPU utilization.
3. Using the `wandb.log` API to log custom metrics.

all using the CIFAR-10 dataset.


In [2]:
!pip install wandb -qU
!pip install tensorflow
!pip install keras

In [2]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tf_keras import layers
from tf_keras import models
from tf_keras.datasets import cifar10
import tf_keras.datasets as tfds

In [7]:
!pip install -qU wandb

In [3]:
import wandb
#from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback
from wandb.integration.keras import WandbMetricsLogger,WandbModelCheckpoint, WandbEvalCallback


wandb.login(key='your_api_key')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: eclipsetheassistant (eclipsetheassistant-svkm-s-narsee-monjee-institute-of-ma). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/dhruvpithadia/.netrc


True

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train[::5] / 255., x_test / 255.
y_train = y_train[::5]

CLASS_NAMES = ["airplane", "automobile", "bird", "cat",
               "deer", "dog", "frog", "horse", "ship", "truck"]

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

Shape of x_train:  (10000, 32, 32, 3)
Shape of y_train:  (10000, 1)
Shape of x_test:  (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [5]:
def Model():
  inputs = keras.layers.Input(shape=(32, 32, 3))

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=2)(x)

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dense(32, activation='relu')(x)

  outputs = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

Give `wandb.init` the `config`

First initialize wandb run, set the hyperparameters.
They're passed in as a dictionary via the `config` argument, and then become available as the `config` attribute of `wandb`.

In [6]:
# Initialize wandb with project name
run = wandb.init(project='wandb-cifar-keras',
                 config={
                     "learning_rate": 0.005,
                     "epochs": 5,
                     "batch_size": 1024,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
model.compile(optimizer, config.loss_function, metrics=['acc'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,322 (145.79 KB)

 Trainable params: 37,322 (145.79 KB)

 Non-trainable params: 0 (0.00 B)

Pass `WandbMetricsLogger` and `WandbModelCheckpoint` to `model.fit`

Keras has a [robust callbacks system](https://keras.io/api/callbacks/) that
allows users to separate model definition and the core training logic
from other behaviors that occur during training and testing.

In [7]:
# Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints
wandb_callbacks = [
    WandbMetricsLogger(),
    WandbModelCheckpoint(filepath="my_model_{epoch:02d}.keras"),
]

model.fit(x_train, y_train,
          epochs=config.epochs,
          batch_size=config.batch_size,
          validation_data=(x_test, y_test),
          callbacks=wandb_callbacks)

Epoch 1/5


Use `wandb.log` for custom metrics

Here, we log the error rate on the test set.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Error Rate: ', round((1 - accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

run.finish()

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - acc: 0.2099 - loss: 1.9768
Test Error Rate:  78.71


Test Error Rate,▁
epoch/acc,▁▅▇▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▆▄▃▁
epoch/val_acc,▁▅▅██
epoch/val_loss,█▆▅▂▁
Test Error Rate,78.71
epoch/acc,0.2084
epoch/epoch,4
epoch/learning_rate,0.005
